In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
#from google.colab import files
#uploaded = files.upload()

In [3]:
#import io
#olist_reviews = pd.read_csv(io.BytesIO(uploaded['olist_order_reviews_dataset.csv']))

In [4]:
# change working directory
os.chdir(r"C:\Users\olani\OneDrive\Documents\Data Science\SMU-Data Science\Machine Learning 1\Olist_Dataset\Olist_Datasets")
#"C:\Users\olani\OneDrive\Documents\Data Science\SMU-Data Science\Machine Learning 1\Olist_Dataset"

# get current working directory
os.getcwd()

'C:\\Users\\olani\\OneDrive\\Documents\\Data Science\\SMU-Data Science\\Machine Learning 1\\Olist_Dataset\\Olist_Datasets'

In [5]:
# set up some colors and text attributes to markdown
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [6]:
olist_reviews = pd.read_csv('olist_order_reviews_dataset.csv')

In [7]:
olist_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [8]:
type(olist_reviews)

pandas.core.frame.DataFrame

In [9]:
olist_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   review_id                100000 non-null  object
 1   order_id                 100000 non-null  object
 2   review_score             100000 non-null  int64 
 3   review_comment_title     11715 non-null   object
 4   review_comment_message   41753 non-null   object
 5   review_creation_date     100000 non-null  object
 6   review_answer_timestamp  100000 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [10]:
olist_rev = olist_reviews.loc[:, ['review_score', 'review_comment_message']]
olist_rev = olist_rev.dropna(subset=['review_comment_message'])
olist_rev = olist_rev.reset_index(drop=True)
print(f'Dataset shape: {olist_rev.shape}')
olist_rev.info()

Dataset shape: (41753, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41753 entries, 0 to 41752
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   review_score            41753 non-null  int64 
 1   review_comment_message  41753 non-null  object
dtypes: int64(1), object(1)
memory usage: 652.5+ KB


### NLP

In [11]:
import nltk
import numpy as np
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import pandas as pd
import re
import matplotlib.pyplot as plt
from pprint import pprint

pd.options.display.max_colwidth = 200
%matplotlib inline

#### Breakline and Carriage Return

In [12]:
def re_breakline(text_list):
    """
    Args:
    ----------
    text_list: list object with text content to be prepared [type: list]
    """
    
    # Applying regex
    return [re.sub('[\n\r]', ' ', r) for r in text_list]

In [13]:
# Creating a list of comment reviews
reviews = list(olist_rev['review_comment_message'].values)

# Applying RegEx
reviews_breakline = re_breakline(reviews)
olist_rev['re_breakline'] = reviews_breakline

# Verifying results
#print(reviews, reviews_breakline, idx_list=[48])
reviews_breakline[:5]
reviews[:5]

['Recebi bem antes do prazo estipulado.',
 'Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa',
 'aparelho eficiente. no site a marca do aparelho esta impresso como 3desinfector e ao chegar esta com outro nome...atualizar com a marca correta uma vez que é o mesmo aparelho',
 'Mas um pouco ,travando...pelo valor ta Boa.\r\n',
 'Vendedor confiável, produto ok e entrega antes do prazo.']

In [14]:
def re_hiperlinks(text_list):
    """
    Args:
    ----------
    text_list: list object with text content to be prepared [type: list]
    """
    
    # Applying regex
    pattern = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return [re.sub(pattern, ' link ', r) for r in text_list]

In [15]:
# Applying RegEx
reviews_hiperlinks = re_hiperlinks(reviews_breakline)
olist_rev['re_hiperlinks'] = reviews_hiperlinks

In [16]:
#Dates
def re_dates(text_list):
    """
    Args:
    ----------
    text_list: list object with text content to be prepared [type: list]
    """
    
    # Applying regex
    pattern = '([0-2][0-9]|(3)[0-1])(\/|\.)(((0)[0-9])|((1)[0-2]))(\/|\.)\d{2,4}'
    return [re.sub(pattern, ' data ', r) for r in text_list]



In [17]:
# Applying RegEx
reviews_dates = re_dates(reviews_hiperlinks)
olist_rev['re_dates'] = reviews_dates

In [18]:
#Money
def re_money(text_list):
    """
    Args:
    ----------
    text_list: list object with text content to be prepared [type: list]
    """
    
    # Applying regex
    pattern = '[R]{0,1}\$[ ]{0,}\d+(,|\.)\d+'
    return [re.sub(pattern, ' dinheiro ', r) for r in text_list]

In [19]:
# Applying RegEx
reviews_money = re_money(reviews_dates)
olist_rev['re_money'] = reviews_money


In [20]:
def re_numbers(text_list):
    """
    Args:
    ----------
    text_series: list object with text content to be prepared [type: list]
    """
    
    # Applying regex
    return [re.sub('[0-9]+', ' numero ', r) for r in text_list]

In [21]:
# Applying RegEx
reviews_numbers = re_numbers(reviews_money)
olist_rev['re_numbers'] = reviews_numbers


Removing negation stop words makes it difficult to see the semantics/sentiment of the comment.
- So we can replace common negation words with another word that indicates negation and that is not in the stopwords list

In [22]:
#Negation
def re_negation(text_list):
    """
    Args:
    ----------
    text_series: list object with text content to be prepared [type: list]
    """
    
    # Applying regex
    return [re.sub('([nN][ãÃaA][oO]|[ñÑ]| [nN] )', ' negação ', r) for r in text_list]

In [23]:
# Applying RegEx
reviews_negation = re_negation(reviews_numbers)
olist_rev['re_negation'] = reviews_negation


In [24]:
#Special Characters
def re_special_chars(text_list):
    """
    Args:
    ----------
    text_series: list object with text content to be prepared [type: list]
    """
    
    # Applying regex
    return [re.sub('\W', ' ', r) for r in text_list]

In [25]:
# Applying RegEx
reviews_special_chars = re_special_chars(reviews_negation)
olist_rev['re_special_chars'] = reviews_special_chars

In [26]:
#Additional whitespaces
def re_whitespaces(text_list):
    """
    Args:
    ----------
    text_series: list object with text content to be prepared [type: list]
    """
    
    # Applying regex
    white_spaces = [re.sub('\s+', ' ', r) for r in text_list]
    white_spaces_end = [re.sub('[ \t]+$', '', r) for r in white_spaces]
    return white_spaces_end

In [27]:
# Applying RegEx
reviews_whitespaces = re_whitespaces(reviews_special_chars)
olist_rev['re_whitespaces'] = reviews_whitespaces

In [28]:
#.drop(columns=['stemming', 'stopwords_removed'], inplace=True)

In [29]:
olist_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41753 entries, 0 to 41752
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   review_score            41753 non-null  int64 
 1   review_comment_message  41753 non-null  object
 2   re_breakline            41753 non-null  object
 3   re_hiperlinks           41753 non-null  object
 4   re_dates                41753 non-null  object
 5   re_money                41753 non-null  object
 6   re_numbers              41753 non-null  object
 7   re_negation             41753 non-null  object
 8   re_special_chars        41753 non-null  object
 9   re_whitespaces          41753 non-null  object
dtypes: int64(1), object(9)
memory usage: 3.2+ MB


In [30]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\olani\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
#removing stop words after replacing negation stop words.
pt_stopwords = stopwords.words('portuguese')
print(f'Total portuguese stopwords in the nltk.corpous module: {len(pt_stopwords)}')
pt_stopwords[:10]

Total portuguese stopwords in the nltk.corpous module: 204


['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um']

In [32]:
# Defining a function to remove the stopwords and to lower the comments
def stopwords_removal(text, cached_stopwords=stopwords.words('portuguese')):
    """
    Args:
    ----------
    text: list object where the stopwords will be removed [type: list]
    cached_stopwords: stopwords to be applied on the process [type: list, default: stopwords.words('portuguese')]
    """
    
    return [c.lower() for c in text.split() if c.lower() not in cached_stopwords]

In [33]:
# Removing stopwords and looking at some examples
reviews_stopwords = [' '.join(stopwords_removal(review)) for review in reviews_whitespaces]
olist_rev['stopwords_removed'] = reviews_stopwords

In [34]:
pip install git+https://github.com/stanfordnlp/stanza.git

Note: you may need to restart the kernel to use updated packages.


'C:\Users\olani\anaconda3\envs\Machine' is not recognized as an internal or external command,
operable program or batch file.


In [35]:
#Alternative lemmatizer for Portuguese
#https://lars76.github.io/2018/05/08/portuguese-lemmatizers.html#7
import stanza
stanza.download('pt')
nlp = stanza.Pipeline('pt')

text = ""
pos = ""
lemma = ""
for sent in nlp("Não, minha miúda no sentido que és como uma irmã para mim.").sentences:
    for word in sent.words:
        text += word.text + "\t"
        pos += word.upos + "\t"
        lemma += word.lemma + "\t"



ModuleNotFoundError: No module named 'stanza'

In [ ]:
nlp = stanza.Pipeline('pt')

text = ""
pos = ""
lemma = ""
for sent in nlp("Não, minha miúda no sentido que és como uma irmã para mim.").sentences:
    for word in sent.words:
        text += word.text + "\t"
        pos += word.upos + "\t"
        lemma += word.lemma + "\t"


In [ ]:
lemma

In [36]:
#Using NLTK Stemmer
# Defining a function to remove the stopwords and to lower the comments
import re
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import joblib


In [37]:
nltk.download('rslp')

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\olani\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [38]:
def stemming_process(text, stemmer=RSLPStemmer()):
    """
    Args:
    ----------
    text: list object where the stopwords will be removed [type: list]
    stemmer: type of stemmer to be applied [type: class, default: RSLPStemmer()]
    """
    
    return [stemmer.stem(c) for c in text.split()]

In [39]:
# Applying stemming and looking at some examples
reviews_stemmer = [' '.join(stemming_process(review)) for review in reviews_stopwords]
olist_rev['stemming'] = reviews_stemmer

In [40]:
olist_rev.iloc[:1, 3:].T

,0
re_hiperlinks,Recebi bem antes do prazo estipulado.
re_dates,Recebi bem antes do prazo estipulado.
re_money,Recebi bem antes do prazo estipulado.
re_numbers,Recebi bem antes do prazo estipulado.
re_negation,Recebi bem antes do prazo estipulado.
re_special_chars,Recebi bem antes do prazo estipulado
re_whitespaces,Recebi bem antes do prazo estipulado
stopwords_removed,recebi bem antes prazo estipulado
stemming,receb bem ant praz estipul


In [41]:
olist_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41753 entries, 0 to 41752
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   review_score            41753 non-null  int64 
 1   review_comment_message  41753 non-null  object
 2   re_breakline            41753 non-null  object
 3   re_hiperlinks           41753 non-null  object
 4   re_dates                41753 non-null  object
 5   re_money                41753 non-null  object
 6   re_numbers              41753 non-null  object
 7   re_negation             41753 non-null  object
 8   re_special_chars        41753 non-null  object
 9   re_whitespaces          41753 non-null  object
 10  stopwords_removed       41753 non-null  object
 11  stemming                41753 non-null  object
dtypes: int64(1), object(11)
memory usage: 3.8+ MB


In [42]:
from sklearn.feature_extraction.text import CountVectorizer

In [43]:
cv = CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(olist_rev['stemming'])
cv_matrix

<41753x8146 sparse matrix of type '<class 'numpy.int64'>'
	with 296360 stored elements in Compressed Sparse Row format>

In [44]:
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [46]:
vocab = cv.get_feature_names()

In [47]:
# show document feature vectors
pd.DataFrame(cv_matrix, columns=vocab)

,__,aa,aaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaguard,aaind,aanch,aaprelh,ab,aba,...,ônibu,ônu,ötim,últ,úmid,únic,úte,útel,útil,ünic
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41748,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41749,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41750,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41751,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
%%time
# you can set the n-gram range to 1,2 to get unigrams as well as bigrams
bv = CountVectorizer(ngram_range=(2,2))
bv_matrix = bv.fit_transform(olist_rev['stemming'])


Wall time: 875 ms


In [49]:
%%time
#System crashed
#bv_matrix = bv_matrix.toarray()
#vocab = bv.get_feature_names()
#pd.DataFrame(bv_matrix, columns=vocab)

MemoryError: Unable to allocate 28.3 GiB for an array with shape (41753, 90926) and data type int64

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [51]:
tv = TfidfVectorizer(min_df=0., max_df=1., norm='l2', use_idf=True, smooth_idf=True)

In [53]:
tv_matrix = tv.fit_transform(olist_rev['stemming'])
tv_matrix = tv_matrix.toarray()
vocab = tv.get_feature_names()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

,__,aa,aaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaguard,aaind,aanch,aaprelh,ab,aba,...,ônibu,ônu,ötim,últ,úmid,únic,úte,útel,útil,ünic
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41748,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41749,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41751,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
from keras.preprocessing import text
from keras.utils import np_utils
from keras.preprocessing import sequence

ModuleNotFoundError: No module named 'keras'

In [55]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(tv_matrix)
similarity_df = pd.DataFrame(similarity_matrix)
#similarity_df

MemoryError: Unable to allocate 2.53 GiB for an array with shape (41753, 8146) and data type float64

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
Z = linkage(similarity_matrix, 'ward')
pd.DataFrame(Z, columns=['Document\Cluster 1', 'Document\Cluster 2', 'Distance', 'Cluster Size'], dtype='object')

In [57]:
olist_rev.head()

,review_score,review_comment_message,re_breakline,re_hiperlinks,re_dates,re_money,re_numbers,re_negation,re_special_chars,re_whitespaces,stopwords_removed,stemming
0,5,Recebi bem antes do prazo estipulado.,Recebi bem antes do prazo estipulado.,Recebi bem antes do prazo estipulado.,Recebi bem antes do prazo estipulado.,Recebi bem antes do prazo estipulado.,Recebi bem antes do prazo estipulado.,Recebi bem antes do prazo estipulado.,Recebi bem antes do prazo estipulado,Recebi bem antes do prazo estipulado,recebi bem antes prazo estipulado,receb bem ant praz estipul
1,5,Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa,Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa,Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa,Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa,Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa,Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa,Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa,Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa,Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa,parabéns lojas lannister adorei comprar internet seguro prático parabéns todos feliz páscoa,parabém loj lannist ador compr internet segur prát parabém tod feliz pásco
2,4,aparelho eficiente. no site a marca do aparelho esta impresso como 3desinfector e ao chegar esta com outro nome...atualizar com a marca correta uma vez que é o mesmo aparelho,aparelho eficiente. no site a marca do aparelho esta impresso como 3desinfector e ao chegar esta com outro nome...atualizar com a marca correta uma vez que é o mesmo aparelho,aparelho eficiente. no site a marca do aparelho esta impresso como 3desinfector e ao chegar esta com outro nome...atualizar com a marca correta uma vez que é o mesmo aparelho,aparelho eficiente. no site a marca do aparelho esta impresso como 3desinfector e ao chegar esta com outro nome...atualizar com a marca correta uma vez que é o mesmo aparelho,aparelho eficiente. no site a marca do aparelho esta impresso como 3desinfector e ao chegar esta com outro nome...atualizar com a marca correta uma vez que é o mesmo aparelho,aparelho eficiente. no site a marca do aparelho esta impresso como numero desinfector e ao chegar esta com outro nome...atualizar com a marca correta uma vez que é o mesmo aparelho,aparelho eficiente. no site a marca do aparelho esta impresso como numero desinfector e ao chegar esta com outro nome...atualizar com a marca correta uma vez que é o mesmo aparelho,aparelho eficiente no site a marca do aparelho esta impresso como numero desinfector e ao chegar esta com outro nome atualizar com a marca correta uma vez que é o mesmo aparelho,aparelho eficiente no site a marca do aparelho esta impresso como numero desinfector e ao chegar esta com outro nome atualizar com a marca correta uma vez que é o mesmo aparelho,aparelho eficiente site marca aparelho impresso numero desinfector chegar outro nome atualizar marca correta vez aparelho,aparelh efici sit marc aparelh impress numer desinfec cheg outr nom atual marc corret vez aparelh
3,4,"Mas um pouco ,travando...pelo valor ta Boa.\r\n","Mas um pouco ,travando...pelo valor ta Boa.","Mas um pouco ,travando...pelo valor ta Boa.","Mas um pouco ,travando...pelo valor ta Boa.","Mas um pouco ,travando...pelo valor ta Boa.","Mas um pouco ,travando...pelo valor ta Boa.","Mas um pouco ,travando...pelo valor ta Boa.",Mas um pouco travando pelo valor ta Boa,Mas um pouco travando pelo valor ta Boa,pouco travando valor ta boa,pouc trav val ta boa
4,5,"Vendedor confiável, produto ok e entrega antes do prazo.","Vendedor co

In [69]:
olist_rev_clean2 = olist_rev[['stemming', 'review_score']]
olist_rev_clean2.head(n=20)

,stemming,review_score
0,receb bem ant praz estipul,5
1,parabém loj lannist ador compr internet segur prát parabém tod feliz pásco,5
2,aparelh efici sit marc aparelh impress numer desinfec cheg outr nom atual marc corret vez aparelh,4
3,pouc trav val ta boa,4
4,vend confi produt ok entreg ant praz,5
5,gost sab sempr receb compr agor decpcion,2
6,péss,1
7,loj not numer,5
8,obrig atença amim dispens,5
9,compr realiz facil entreg efetu ant praz dad produt começ ser us pres problem,5


In [70]:
# let's break up the Discrepancy variable
#But needs outliers to be removed
olist_rev_clean2['review_score_class'] = pd.cut(olist_rev_clean2['review_score'],
                                                 [0, 2, 3, 5],
                                                 3,
                                                 labels=['bad','fair','good']) # this creates a new variable
olist_rev_clean2['review_score_class'] = olist_rev_clean2.copy()['review_score_class'].astype("category")
olist_rev_clean2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41753 entries, 0 to 41752
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   stemming            41753 non-null  object  
 1   review_score        41753 non-null  int64   
 2   review_score_class  41753 non-null  category
dtypes: category(1), int64(1), object(1)
memory usage: 693.4+ KB


C:\Users\olani\anaconda3\envs\Machine Learning 1\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\olani\anaconda3\envs\Machine Learning 1\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [71]:
olist_rev_clean2.head(n=20)

,stemming,review_score,review_score_class
0,receb bem ant praz estipul,5,good
1,parabém loj lannist ador compr internet segur prát parabém tod feliz pásco,5,good
2,aparelh efici sit marc aparelh impress numer desinfec cheg outr nom atual marc corret vez aparelh,4,good
3,pouc trav val ta boa,4,good
4,vend confi produt ok entreg ant praz,5,good
5,gost sab sempr receb compr agor decpcion,2,bad
6,péss,1,bad
7,loj not numer,5,good
8,obrig atença amim dispens,5,good
9,compr realiz facil entreg efetu ant praz dad produt começ ser us pres problem,5,good


In [72]:
olist_rev_clean3 = olist_rev_clean2.copy().drop(['review_score'], axis=1) # axis=1 means column
olist_rev_clean3.head()

,stemming,review_score_class
0,receb bem ant praz estipul,good
1,parabém loj lannist ador compr internet segur prát parabém tod feliz pásco,good
2,aparelh efici sit marc aparelh impress numer desinfec cheg outr nom atual marc corret vez aparelh,good
3,pouc trav val ta boa,good
4,vend confi produt ok entreg ant praz,good


In [73]:
olist_rev_clean3

,stemming,review_score_class
0,receb bem ant praz estipul,good
1,parabém loj lannist ador compr internet segur prát parabém tod feliz pásco,good
2,aparelh efici sit marc aparelh impress numer desinfec cheg outr nom atual marc corret vez aparelh,good
3,pouc trav val ta boa,good
4,vend confi produt ok entreg ant praz,good
...,...,...
41748,entreg dentr praz produt cheg cond perfeit satisfeit,good
41749,produt neg envi nf neg exist vend nf cert fic aguard envi nf pod ser mail,fair
41750,excel mochil entreg sup rápid sup recom loj,good
41751,solicit compr cap retrovi celt prism meriv pret lad esquerd cheg mim cap lad direit outr model carr decepcion fic aguard,bad


In [ ]:
comments_stemmed = []
for stem in 

In [74]:
reviews_stopwords[:15]

['recebi bem antes prazo estipulado',
 'parabéns lojas lannister adorei comprar internet seguro prático parabéns todos feliz páscoa',
 'aparelho eficiente site marca aparelho impresso numero desinfector chegar outro nome atualizar marca correta vez aparelho',
 'pouco travando valor ta boa',
 'vendedor confiável produto ok entrega antes prazo',
 'gostaria saber sempre recebi compra agora decpcionou',
 'péssimo',
 'loja nota numero',
 'obrigado atençao amim dispensada',
 'compra realizada facilmente entrega efetuada antes prazo dado produto começou ser usado presente problemas',
 'relógio bonito barato',
 'negação gostei comprei gato lebre',
 'sempre compro internet entrega ocorre antes prazo combinado acredito ser prazo máximo stark prazo máximo esgotou ainda negação recebi produto',
 'recebi exatamente esperava demais encomendas outros vendedores atrasaram chegou prazo',
 'recomendo']

In [75]:
import googletrans
from googletrans import Translator

In [76]:
translator = Translator()

In [ ]:
for words in reviews_stopwords:
    

In [79]:
rev_translations = []
for word in reviews_stopwords:
    rev_translations.append(translator.translate(word).text)
#reviews_eng = reviews_stopwords.apply(translator.translate)

KeyboardInterrupt: 

In [ ]:
rev_translations[:8]

In [81]:
# let's break up the Discrepancy variable
#But needs outliers to be removed
import sys
olist_rev_clean2['review_sentiment'] = pd.cut(olist_rev_clean2['review_score'],
                                                 [0, 3, 5],
                                                 2,
                                                 labels=['negative','positive']) # this creates a new variable
olist_rev_clean2['review_sentiment'] = olist_rev_clean2.copy()['review_sentiment'].astype("category")
olist_rev_clean2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41753 entries, 0 to 41752
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   stemming            41753 non-null  object  
 1   review_score        41753 non-null  int64   
 2   review_score_class  41753 non-null  category
 3   review_sentiment    41753 non-null  category
dtypes: category(2), int64(1), object(1)
memory usage: 734.3+ KB


C:\Users\olani\anaconda3\envs\Machine Learning 1\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\olani\anaconda3\envs\Machine Learning 1\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [82]:
olist_rev_clean4 = olist_rev_clean2.copy().drop(columns=['review_score', 'review_score_class']) 
olist_rev_clean4.head()

,stemming,review_sentiment
0,receb bem ant praz estipul,positive
1,parabém loj lannist ador compr internet segur prát parabém tod feliz pásco,positive
2,aparelh efici sit marc aparelh impress numer desinfec cheg outr nom atual marc corret vez aparelh,positive
3,pouc trav val ta boa,positive
4,vend confi produt ok entreg ant praz,positive


In [83]:
olist_rev_clean4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41753 entries, 0 to 41752
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   stemming          41753 non-null  object  
 1   review_sentiment  41753 non-null  category
dtypes: category(1), object(1)
memory usage: 367.2+ KB


In [85]:
olist_rev_clean4.shape[0]*0.85

35490.049999999996

In [88]:
#Stratified shuffle split for representative dataset.

from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(olist_rev_clean4, olist_rev_clean4["review_sentiment"]):
    strat_train_set = olist_rev_clean4.loc[train_index]
    strat_test_set = olist_rev_clean4.loc[test_index]

In [89]:
strat_train_set.head(), strat_train_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33402 entries, 25624 to 32688
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   stemming          33402 non-null  object  
 1   review_sentiment  33402 non-null  category
dtypes: category(1), object(1)
memory usage: 554.6+ KB


(                                               stemming review_sentiment
 25624                                         bom recom         positive
 5654     bom mor nort sempr dem cheg cois aqu bem rapid         positive
 20891                                   bom compr stark         positive
 28633  produt neg entreg loj neg deu satisf após contat         negative
 5099                produt neg entreg dar satisf client         negative,
 None)

In [93]:
X_train = np.array(strat_train_set["stemming"])

In [94]:
y_train = np.array(strat_train_set["review_sentiment"])
X_test = np.array(strat_test_set["stemming"]) 
y_test = np.array(strat_test_set["review_sentiment"])

In [95]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [108]:
#Model evaluation metric
# -*- coding: utf-8 -*-
"""
Created on Mon Jul 31 20:05:23 2017

@author: DIP
@Copyright: Dipanjan Sarkar
"""

from sklearn import metrics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.base import clone
from sklearn.preprocessing import label_binarize
from scipy import interp
from sklearn.metrics import roc_curve, auc 


def get_metrics(true_labels, predicted_labels):
    
    print('Accuracy:', np.round(
                        metrics.accuracy_score(true_labels, 
                                               predicted_labels),
                        4))
    print('Precision:', np.round(
                        metrics.precision_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
    print('Recall:', np.round(
                        metrics.recall_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
    print('F1 Score:', np.round(
                        metrics.f1_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
                        

def train_predict_model(classifier, 
                        train_features, train_labels, 
                        test_features, test_labels):
    # build model    
    classifier.fit(train_features, train_labels)
    # predict using model
    predictions = classifier.predict(test_features) 
    return predictions    


def display_confusion_matrix(true_labels, predicted_labels, classes=[1,0]):
    
    total_classes = len(classes)
    level_labels = [total_classes*[0], list(range(total_classes))]

    cm = metrics.confusion_matrix(y_true=true_labels, y_pred=predicted_labels, 
                                  labels=classes)
    cm_frame = pd.DataFrame(data=cm, 
                            columns=pd.MultiIndex(levels=[['Predicted:'], classes], 
                                                  labels=level_labels), 
                            index=pd.MultiIndex(levels=[['Actual:'], classes], 
                                                labels=level_labels)) 
    print(cm_frame) 


def display_confusion_matrix_pretty(true_labels, predicted_labels, classes=[1,0]):
    
    total_classes = len(classes)
    level_labels = [total_classes*[0], list(range(total_classes))]

    cm = metrics.confusion_matrix(y_true=true_labels, y_pred=predicted_labels, 
                                  labels=classes)
    cm_frame = pd.DataFrame(data=cm, 
                            columns=pd.MultiIndex(levels=[['Predicted:'], classes], 
                                                  labels=level_labels), 
                            index=pd.MultiIndex(levels=[['Actual:'], classes], 
                                                labels=level_labels)) 
    return cm_frame
    
def display_classification_report(true_labels, predicted_labels, classes=[1,0]):

    report = metrics.classification_report(y_true=true_labels, 
                                           y_pred=predicted_labels, 
                                           labels=classes) 
    print(report)
    
    
    
def display_model_performance_metrics(true_labels, predicted_labels, classes=[1,0]):
    print('Model Performance metrics:')
    print('-'*30)
    get_metrics(true_labels=true_labels, predicted_labels=predicted_labels)
    print('\nModel Classification report:')
    print('-'*30)
    display_classification_report(true_labels=true_labels, predicted_labels=predicted_labels, 
                                  classes=classes)
    print('\nPrediction Confusion Matrix:')
    print('-'*30)
    display_confusion_matrix(true_labels=true_labels, predicted_labels=predicted_labels, 
                             classes=classes)


def plot_model_decision_surface(clf, train_features, train_labels,
                                plot_step=0.02, cmap=plt.cm.RdYlBu,
                                markers=None, alphas=None, colors=None):
    
    if train_features.shape[1] != 2:
        raise ValueError("X_train should have exactly 2 columnns!")
    
    x_min, x_max = train_features[:, 0].min() - plot_step, train_features[:, 0].max() + plot_step
    y_min, y_max = train_features[:, 1].min() - plot_step, train_features[:, 1].max() + plot_step
    xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))

    clf_est = clone(clf)
    clf_est.fit(train_features,train_labels)
    if hasattr(clf_est, 'predict_proba'):
        Z = clf_est.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:,1]
    else:
        Z = clf_est.predict(np.c_[xx.ravel(), yy.ravel()])    
    Z = Z.reshape(xx.shape)
    cs = plt.contourf(xx, yy, Z, cmap=cmap)
    
    le = LabelEncoder()
    y_enc = le.fit_transform(train_labels)
    n_classes = len(le.classes_)
    plot_colors = ''.join(colors) if colors else [None] * n_classes
    label_names = le.classes_
    markers = markers if markers else [None] * n_classes
    alphas = alphas if alphas else [None] * n_classes
    for i, color in zip(range(n_classes), plot_colors):
        idx = np.where(y_enc == i)
        plt.scatter(train_features[idx, 0], train_features[idx, 1], c=color,
                    label=label_names[i], cmap=cmap, edgecolors='black', 
                    marker=markers[i], alpha=alphas[i])
    plt.legend()
    plt.show()


def plot_model_roc_curve(clf, features, true_labels, label_encoder=None, class_names=None):
    
    ## Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    if hasattr(clf, 'classes_'):
        class_labels = clf.classes_
    elif label_encoder:
        class_labels = label_encoder.classes_
    elif class_names:
        class_labels = class_names
    else:
        raise ValueError('Unable to derive prediction classes, please specify class_names!')
    n_classes = len(class_labels)
    y_test = label_binarize(true_labels, classes=class_labels)
    if n_classes == 2:
        if hasattr(clf, 'predict_proba'):
            prob = clf.predict_proba(features)
            y_score = prob[:, prob.shape[1]-1] 
        elif hasattr(clf, 'decision_function'):
            prob = clf.decision_function(features)
            y_score = prob[:, prob.shape[1]-1]
        else:
            raise AttributeError("Estimator doesn't have a probability or confidence scoring system!")
        
        fpr, tpr, _ = roc_curve(y_test, y_score)      
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label='ROC curve (area = {0:0.2f})'
                                 ''.format(roc_auc),
                 linewidth=2.5)
        
    elif n_classes > 2:
        if hasattr(clf, 'predict_proba'):
            y_score = clf.predict_proba(features)
        elif hasattr(clf, 'decision_function'):
            y_score = clf.decision_function(features)
        else:
            raise AttributeError("Estimator doesn't have a probability or confidence scoring system!")

        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        ## Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        ## Compute macro-average ROC curve and ROC area
        # First aggregate all false positive rates
        all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
        # Then interpolate all ROC curves at this points
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(n_classes):
            mean_tpr += interp(all_fpr, fpr[i], tpr[i])
        # Finally average it and compute AUC
        mean_tpr /= n_classes
        fpr["macro"] = all_fpr
        tpr["macro"] = mean_tpr
        roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

        ## Plot ROC curves
        plt.figure(figsize=(6, 4))
        plt.plot(fpr["micro"], tpr["micro"],
                 label='micro-average ROC curve (area = {0:0.2f})'
                       ''.format(roc_auc["micro"]), linewidth=3)

        plt.plot(fpr["macro"], tpr["macro"],
                 label='macro-average ROC curve (area = {0:0.2f})'
                       ''.format(roc_auc["macro"]), linewidth=3)

        for i, label in enumerate(class_labels):
            plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                           ''.format(label, roc_auc[i]), 
                     linewidth=2, linestyle=':')
    else:
        raise ValueError('Number of classes should be atleast 2 or more')
        
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()




In [113]:
# change working directory to where I could locate the model_eval_util function
os.chdir(r"C:\Users\olani\OneDrive\Documents\Data Science\SMU-Data Science\Natural Language Processing\Text Analytics with Python_Dipanjan 2019\Codes_Dipanjan 2019\Ch05 - Text Classification")
#"C:\Users\olani\OneDrive\Documents\Data Science\SMU-Data Science\Machine Learning 1\Olist_Dataset"

# get current working directory
os.getcwd()

'C:\\Users\\olani\\OneDrive\\Documents\\Data Science\\SMU-Data Science\\Natural Language Processing\\Text Analytics with Python_Dipanjan 2019\\Codes_Dipanjan 2019\\Ch05 - Text Classification'

In [114]:
import model_evaluation_utils as meu

In [107]:
# build BOW features on train reviews

cv = CountVectorizer(binary=False, min_df=0.0, max_df=1.0, ngram_range=(1,2))
cv_train_features = cv.fit_transform(X_train)

In [97]:
# build TFIDF features on train reviews
tv = TfidfVectorizer(use_idf=True, min_df=0.0, max_df=1.0, ngram_range=(1,2), sublinear_tf=True)
tv_train_features = tv.fit_transform(X_train)

In [98]:
# transform test reviews into features
cv_test_features = cv.transform(X_test)
tv_test_features = tv.transform(X_test)

In [99]:
print('BOW model:> Train features shape:', cv_train_features.shape,' Test features shape:', cv_test_features.shape)

BOW model:> Train features shape: (33402, 84440)  Test features shape: (8351, 84440)


In [100]:
print('TFIDF model:> Train features shape:', tv_train_features.shape, ' Test features shape:', tv_test_features.shape)

TFIDF model:> Train features shape: (33402, 84440)  Test features shape: (8351, 84440)


In [101]:
from sklearn.linear_model import SGDClassifier, LogisticRegression

In [102]:
lr = LogisticRegression(penalty='l2', max_iter=100, C=1)
svm = SGDClassifier(loss='hinge', max_iter=100)

In [115]:
lr_bow_predictions = meu.train_predict_model(classifier=lr,train_features=cv_train_features, 
                                         train_labels=y_train, test_features=cv_test_features, test_labels=y_test)

C:\Users\olani\anaconda3\envs\Machine Learning 1\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [116]:
meu.display_model_performance_metrics(true_labels=y_test,predicted_labels=lr_bow_predictions,
                                      classes=['positive', 'negative'])

Model Performance metrics:
------------------------------
Accuracy: 0.8991
Precision: 0.8985
Recall: 0.8991
F1 Score: 0.8985

Model Classification report:
------------------------------
              precision    recall  f1-score   support

    positive       0.91      0.93      0.92      5336
    negative       0.88      0.84      0.86      3015

    accuracy                           0.90      8351
   macro avg       0.89      0.89      0.89      8351
weighted avg       0.90      0.90      0.90      8351


Prediction Confusion Matrix:
------------------------------


TypeError: __new__() got an unexpected keyword argument 'labels'

In [117]:
# Logistic Regression model on TF-IDF features
lr_tfidf_predictions = meu.train_predict_model(classifier=lr,train_features=tv_train_features, 
                                         train_labels=y_train, test_features=tv_test_features, test_labels=y_test)

C:\Users\olani\anaconda3\envs\Machine Learning 1\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [118]:
meu.display_model_performance_metrics(true_labels=y_test, predicted_labels=lr_tfidf_predictions,
                                      classes=['positive', 'negative'])

Model Performance metrics:
------------------------------
Accuracy: 0.8997
Precision: 0.8999
Recall: 0.8997
F1 Score: 0.8998

Model Classification report:
------------------------------
              precision    recall  f1-score   support

    positive       0.92      0.92      0.92      5336
    negative       0.86      0.87      0.86      3015

    accuracy                           0.90      8351
   macro avg       0.89      0.89      0.89      8351
weighted avg       0.90      0.90      0.90      8351


Prediction Confusion Matrix:
------------------------------


TypeError: __new__() got an unexpected keyword argument 'labels'

In [119]:
from skater.core.local_interpretation.lime.lime_text import LimeTextExplainer

ModuleNotFoundError: No module named 'skater'